In [206]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

**drop unimportant column**

In [207]:
house_data = pd.read_csv('Bengaluru_House_Data.csv')
#house_data.head(4)

In [208]:
house_data.drop(['area_type' , 'availability','society','balcony'] , axis=1 , inplace=True)

In [209]:
house_data.isna().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [210]:
house_data.dropna(inplace=True)

In [211]:
#house_data.isna().sum()

In [212]:
#house_data.head(3)

In [213]:
house_data.shape

(13246, 5)

In [214]:
house_data['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [215]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13246 entries, 0 to 13319
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    13246 non-null  object 
 1   size        13246 non-null  object 
 2   total_sqft  13246 non-null  object 
 3   bath        13246 non-null  float64
 4   price       13246 non-null  float64
dtypes: float64(2), object(3)
memory usage: 620.9+ KB


**convert object columns  to numric ones type**

In [216]:
house_data.bath.unique()

array([ 2.,  5.,  3.,  4.,  6.,  1.,  9.,  8.,  7., 11., 10., 14., 27.,
       12., 16., 40., 15., 13., 18.])

In [217]:
# bath column
house_data.bath.astype('float32')

0        2.0
1        5.0
2        2.0
3        3.0
4        2.0
        ... 
13315    4.0
13316    5.0
13317    2.0
13318    4.0
13319    1.0
Name: bath, Length: 13246, dtype: float32

In [218]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [219]:
#show data that can not be convert to float easily
house_data[~ house_data['total_sqft'].apply(is_float)]

,location,size,total_sqft,bath,price
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800
...,...,...,...,...,...
12975,Whitefield,2 BHK,850 - 1060,2.0,38.190
12990,Talaghattapura,3 BHK,1804 - 2273,3.0,122.000
13059,Harlur,2 BHK,1200 - 1470,2.0,72.760
13265,Hoodi,2 BHK,1133 - 1384,2.0,59.135


In [220]:
def edit_total_sqft(y):
  range_nums = y.split('-')
  if len(range_nums) ==2:
    mean_num = (float(range_nums[0]) + float(range_nums[1])) / 2
    return mean_num
  try:
      return float(y)
  except:
    return None

In [221]:
df_house = house_data.copy()
df_house['total_sqft'] = df_house['total_sqft'].apply(edit_total_sqft)

In [222]:
df_house.dropna(inplace=True)

In [223]:
def edit_size(z):
  num = z.split()
  return float(num[0])

In [224]:
bedroom = df_house['size'].apply(edit_size)
df_house['bedroom'] = bedroom

In [225]:
df_house.head()
df_house.isna().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
bedroom       0
dtype: int64

In [226]:
df_house.head()

,location,size,total_sqft,bath,price,bedroom
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2.0
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4.0
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3.0
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3.0
4,Kothanur,2 BHK,1200.0,2.0,51.00,2.0


In [227]:
price_per_square = (df_house.price * 30000 / df_house.total_sqft)
df_house['price_sqft'] = price_per_square 

**handling unbalanced and  outliers** 

In [228]:
df_house[df_house.bath == 40]

,location,size,total_sqft,bath,price,bedroom,price_sqft
4684,Munnekollal,43 Bedroom,2400.0,40.0,660.0,43.0,8250.0


In [229]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        mean_price = np.mean(subdf.price_sqft)
        strandard_deviation = np.std(subdf.price_sqft)
        reduced_df = subdf[(subdf.price_sqft>(mean_price-strandard_deviation)) & (subdf.price_sqft<=(mean_price+strandard_deviation))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

In [230]:
df_house2 = remove_pps_outliers(df_house)

In [231]:
df_house2.head()

,location,size,total_sqft,bath,price,bedroom,price_sqft
0,Devarabeesana Halli,3 BHK,1672.0,3.0,150.0,3.0,2691.387560
1,Devarabeesana Halli,3 BHK,1750.0,3.0,149.0,3.0,2554.285714
2,Devarabeesana Halli,3 BHK,1750.0,3.0,150.0,3.0,2571.428571
3,Devarachikkanahalli,3 BHK,1250.0,2.0,44.0,3.0,1056.000000
4,Devarachikkanahalli,2 BHK,1250.0,2.0,40.0,2.0,960.000000


In [232]:
def remove_bedroom_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bedroom'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_sqft),
                'std': np.std(bhk_df.price_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bedroom'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

In [233]:
df_house3 = remove_bedroom_outliers(df_house2)

In [234]:
df_house3.head()

,location,size,total_sqft,bath,price,bedroom,price_sqft
0,Devarabeesana Halli,3 BHK,1672.0,3.0,150.0,3.0,2691.387560
1,Devarabeesana Halli,3 BHK,1750.0,3.0,149.0,3.0,2554.285714
2,Devarabeesana Halli,3 BHK,1750.0,3.0,150.0,3.0,2571.428571
4,Devarachikkanahalli,2 BHK,1250.0,2.0,40.0,2.0,960.000000
5,Devarachikkanahalli,2 Bedroom,1200.0,2.0,83.0,2.0,2075.000000


In [235]:
# handle location data
loc_stat = df_house3['location'].value_counts()
loc_less_than_10 = loc_stat[loc_stat <= 10]
df_house['location'] = df_house3['location'].apply(lambda z : 'other' if z in loc_less_than_10  else z)

numric_addres = pd.get_dummies(df_house3['location'])
numric_house_df = pd.concat([df_house3,numric_addres] , axis='columns')

In [236]:
numric_house_df.head()

,location,size,total_sqft,bath,price,bedroom,price_sqft,Devarabeesana Halli,Devarachikkanahalli,Electronic City,Mysore Highway,Rachenahalli,Thanisandra,south,1st Block BEL Layout,1st Block HBR Layout,1st Block HRBR Layout,1st Block Jayanagar,1st Block Koramangala,1st Phase JP Nagar,1st Stage Indira Nagar,2nd Block Hrbr Layout,2nd Block Jayanagar,2nd Block Koramangala,2nd Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Arekere Mico Layout,2nd Stage Nagarbhavi,3rd Block Banashankari,3rd Block Hrbr Layout,3rd Block Jayanagar,3rd Block Koramangala,3rd Phase JP Nagar,4th Block Jayanagar,4th Block Koramangala,4th Phase JP Nagar,4th T block Jayanagar,5th Block Hbr Layout,5th Block Jayanagar,5th Phase JP Nagar,...,Venkatadri Layout,Venkatapura,Venugopal Reddy Layout,Vibhutipura Extension,Vibuthipura,Victoria Layout,Vidyaranyapura,Vignana Nagar,Vijaya Bank Layout,Vijayanagar,Vijinapura,Vimanapura,Vinayak Nagar,Vinayaka Nagar,Virat Nagar,Virupakshapura,Vishveshwarya Layout,Vishwanatha Nagenahalli,Vishwapriya Layout,Vishwapriya Nagar,Vittal Nagar,Vittasandra,Vivek Nagar,Volagerekallahalli,Vyalikaval,Weavers Colony,Whitefield,"Whitefield,",Wilson Garden,Yarandahalli,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yemlur,Yeshwanthpur,Yeshwanthpur Industrial Suburb,cooketown,manyata park,tc.palya
0,Devarabeesana Halli,3 BHK,1672.0,3.0,150.0,3.0,2691.387560,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Devarabeesana Halli,3 BHK,1750.0,3.0,149.0,3.0,2554.285714,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Devarabeesana Halli,3 BHK,1750.0,3.0,150.0,3.0,2571.428571,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Devarachikkanahalli,2 BHK,1250.0,2.0,40.0,2.0,960.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Devarachikkanahalli,2 Bedroom,1200.0,2.0,83.0,2.0,2075.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**split data**

In [237]:
X = numric_house_df.drop(['location','size','price'] , axis="columns")
Y = numric_house_df.price

**Hyperparameter tuning** 

In [238]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import ShuffleSplit
def find_best_model_using_gridsearchcv(X,Y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,Y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])
find_best_model_using_gridsearchcv(X,Y)

,model,best_score,best_params
0,linear_regression,-1.148463e+15,{'normalize': False}
1,lasso,-1.627178e+00,"{'alpha': 2, 'selection': 'cyclic'}"
2,decision_tree,9.669598e-01,"{'criterion': 'mse', 'splitter': 'best'}"


In [239]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X,Y , test_size=0.15)

**bulid the model**

In [240]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(criterion='mse' , splitter='best')
reg.fit(x_train , y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [241]:
reg.score(x_test , y_test)

0.9929727587740512

In [242]:
y_hat = reg.predict(x_test)
y_hat[:5]

array([ 89.,  38.,  57.,  95., 160.])

In [243]:
y_test[:5]

3702     92.0
8170     38.0
602      60.0
3693     97.0
5711    160.0
Name: price, dtype: float64

**save model** 

In [244]:
pip install pickle5


In [245]:
import pickle5

In [247]:
filename = 'prediction_houses_model.sav'
pickle5.dump(reg, open(filename, 'wb'))